In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [6]:
print(df.columns)


Index(['clean_text', 'category'], dtype='object')


In [7]:
print(df.head())

                                          clean_text  category
0  when modi promised “minimum government maximum...      -1.0
1  talk all the nonsense and continue all the dra...       0.0
2  what did just say vote for modi  welcome bjp t...       1.0
3  asking his supporters prefix chowkidar their n...       1.0
4  answer who among these the most powerful world...       1.0


In [11]:
import numpy as np
import pandas as pd
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

nltk.download('stopwords')

# Load data
file_path = "C:\\Users\\HOME\\OneDrive\\Documents\\Downloads\\Twitter_Data.csv"
df = pd.read_csv(file_path)
df = df.dropna()

# Text normalization function
def text_normalize(text):
    text = text.lower()
    text = re.sub(r'^rt[\s]+', '', text)
    text = re.sub(r'https?://.*[\r\n]*', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    stop_words = set(stopwords.words('english'))
    words = [word for word in text.split() if word not in stop_words]
    stemmer = SnowballStemmer('english')
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

# Apply text normalization to the 'clean_text' column
df['clean_text'] = df['clean_text'].apply(text_normalize)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HOME\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
vectorizer = TfidfVectorizer(max_features=2000)
X = vectorizer.fit_transform(df['clean_text']).toarray()

# Add bias to X
intercept = np.ones((X.shape[0], 1))
X_b = np.concatenate((intercept, X), axis=1)


In [14]:
n_classes = df['category'].nunique()
n_samples = df['category'].size
y = df['category'].to_numpy()
y_encoded = np.zeros((n_samples, n_classes))


In [15]:
val_size = 0.2
test_size = 0.125
random_state = 2
is_shuffle = True

X_train, X_val, y_train, y_val = train_test_split(
    X_b, y_encoded, test_size=val_size, random_state=random_state, shuffle=is_shuffle
)
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, test_size=test_size, random_state=random_state, shuffle=is_shuffle
)


In [16]:
def softmax(z):
    exp_z = np.exp(z)
    return exp_z / exp_z.sum(axis=1, keepdims=True)

def predict(X, theta):
    z = np.dot(X, theta)
    return softmax(z)

def compute_loss(y_hat, y):
    n = y.size
    return (-1 / n) * np.sum(y * np.log(y_hat))

def compute_gradient(X, y, y_hat):
    n = y.size
    return np.dot(X.T, (y_hat - y)) / n

def update_theta(theta, gradient, lr):
    return theta - lr * gradient

def compute_accuracy(X, y, theta):
    y_hat = predict(X, theta)
    return (np.argmax(y_hat, axis=1) == np.argmax(y, axis=1)).mean()


In [17]:
lr = 0.1
epochs = 200
batch_size = X_train.shape[0]
n_features = X_train.shape[1]

np.random.seed(random_state)
theta = np.random.uniform(size=(n_features, n_classes))


In [19]:
# Train the model
for epoch in range(epochs):
    # Forward pass: Compute predictions
    y_hat = predict(X_train, theta)
    
    # Calculate loss
    train_loss = compute_loss(y_hat, y_train)
    
    # Compute gradient
    gradient = compute_gradient(X_train, y_train, y_hat)
    
    # Update weights
    theta = update_theta(theta, gradient, lr)
    
    # Optional: Print the training loss every few epochs
    if (epoch + 1) % 10 == 0 or epoch == 0:
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {train_loss:.4f}")


Epoch 1/200, Loss: -0.0000
Epoch 10/200, Loss: -0.0000
Epoch 20/200, Loss: -0.0000
Epoch 30/200, Loss: -0.0000
Epoch 40/200, Loss: -0.0000
Epoch 50/200, Loss: -0.0000
Epoch 60/200, Loss: -0.0000
Epoch 70/200, Loss: -0.0000
Epoch 80/200, Loss: -0.0000
Epoch 90/200, Loss: -0.0000
Epoch 100/200, Loss: -0.0000
Epoch 110/200, Loss: -0.0000
Epoch 120/200, Loss: -0.0000
Epoch 130/200, Loss: -0.0000
Epoch 140/200, Loss: -0.0000
Epoch 150/200, Loss: -0.0000
Epoch 160/200, Loss: -0.0000
Epoch 170/200, Loss: -0.0000
Epoch 180/200, Loss: -0.0000
Epoch 190/200, Loss: -0.0000
Epoch 200/200, Loss: -0.0000


In [20]:
# Evaluate model on validation and test sets
val_set_acc = compute_accuracy(X_val, y_val, theta)
test_set_acc = compute_accuracy(X_test, y_test, theta)
print(f'Validation Accuracy: {val_set_acc:.4f}')
print(f'Test Accuracy: {test_set_acc:.4f}')


Validation Accuracy: 0.3551
Test Accuracy: 0.3543
